In [1]:
import numpy as np
import pandas as pd

import os

In [2]:
monthly_data = pd.read_csv(os.path.join(os.getcwd(),
                                        'data',
                                        'input',
                                        'OR_Databank_Q1_2007-Q4_2024.csv'),
                            header=0)

monthly_data.head(1)

C:\Users\ellin\AppData\Local\Temp\ipykernel_32928\1384618058.py:1: DtypeWarning: Columns (10,15,28,33,34,39,40,41,42,43,44,45,58,59,62,63,64,65,66,67,68,70,72,73,75,77,94,98,99,100,101,103,110,111,112,113,114,115,130,132,147,158,160,161,162,164,166,167,168) have mixed types. Specify dtype option on import or set low_memory=False.
  monthly_data = pd.read_csv(os.path.join(os.getcwd(),


,AHA ID,Hospital Name,Hospital Short Name,Type,Critical Access,Month,Quarter,Year,Available Beds,Licensed Beds,...,Gross Patient Accounts Receivable (Medicare),Gross Patient Accounts Receivable (Medicaid),Gross Patient Accounts Receivable (Self Pay),Gross Patient Accounts Receivable (Commercial),Gross Patient Accounts Receivable (Others),Gross Patient Accounts Receivable,Uncompensated Care,Inpatient Discharges,Total Operating Revenue,Total Revenue
0,6920770,Adventist Health Columbia Gorge Medical Center,Adventist Columbia Gorge Med Ctr,B,False,1,1,2024,43,49,...,"6,312,933","3,151,689","4,301,845","18,645,968","1,363,000","33,775,435",406239,136,9234294,9198710


In [3]:
categorical_cols = [
    'Hospital Name',
    'Hospital Short Name',
    'Type',
    'Critical Access',
] # excluding AHA Id because its a grouping column
proportion_cols = [
    r'Operating Margin %',
    r'total margin %',
    r'CCR',
    r'charity care %',
    r'bad debt %',
    r'Medicaid %',
    r'Medicare %',
    r'Commercial %',
    r'Self Pay %',
]

sum_agg_cols = list(set(monthly_data.columns).difference(set(categorical_cols+proportion_cols+['Month'])))

In [4]:
for col in sum_agg_cols:
    print(f'{col}: {monthly_data[col].dtype}')

Patient Days of DPU: object
Patient Days of Acute Inpatient: object
Bad Debt (Medicare): object
Discharges of Acute Inpatient: object
Patient Days of Swing Bed (Self Pay): int64
Discharges of Acute Inpatient (Commercial): float64
Charity Care (Medicare): object
Capitated Payments (Others): object
Charges of Acute Outpatient (Medicare): object
Patient Days of DPU (Others): object
Contractuals of Acute (Medicare): object
Contractuals of Other (Medicare): object
Discharges of Subacute & LTC (Commercial): float64
Facility Payroll Amount: object
Gross Patient Accounts Receivable (Self Pay): object
Total Revenue: int64
Total Contractual Allowances (Self Pay): object
Non Operating Revenue: object
Contractuals of Other: object
Operating Margin: object
Other Expense: object
Net Patient Revenue: object
Capitated Payments (Commercial): object
Discharges of Subacute & LTC (Self Pay): int64
AHA ID: int64
Patient Days of Swing Bed: int64
Charges of Swing Bed (Medicare): object
Discharges of Subacute

In [5]:
print(monthly_data['Charges of Swing Bed (Medicaid)'].unique())

['0' '36,892' '4,133' ... '2703' '1485' '9827']


In [6]:
monthly_data[sum_agg_cols] = monthly_data[sum_agg_cols].apply(pd.to_numeric, errors='coerce')
print((monthly_data.isna().sum() / monthly_data.shape[0])[(monthly_data.isna().sum() / monthly_data.shape[0]) > 0.0])

Discharges of Acute Inpatient (Medicare)          0.002817
Discharges of Acute Inpatient (Commercial)        0.775569
Discharges of Acute Inpatient                     0.029658
Discharges of Swing Bed (Commercial)              0.775569
Discharges of Subacute & LTC (Commercial)         0.775569
                                                    ...   
Gross Patient Accounts Receivable (Medicaid)      0.196494
Gross Patient Accounts Receivable (Self Pay)      0.196338
Gross Patient Accounts Receivable (Commercial)    0.966195
Gross Patient Accounts Receivable (Others)        0.179905
Gross Patient Accounts Receivable                 0.196494
Length: 153, dtype: float64


In [7]:
agg_dict = {key:'sum' for key in sum_agg_cols}
agg_dict.update({key:(lambda s: s.mode().iloc[0]) for key in categorical_cols})
# agg_dict.update({key:'mean' for key in proportion_cols})

quarterly_sums = monthly_data\
    .drop(columns=['Month'])\
    .groupby(['AHA ID', 'Year', 'Quarter']).agg(agg_dict).reset_index(allow_duplicates=True)

quarterly_sums.head()

,AHA ID,Year,Quarter,Patient Days of DPU,Patient Days of Acute Inpatient,Bad Debt (Medicare),Discharges of Acute Inpatient,Patient Days of Swing Bed (Self Pay),Discharges of Acute Inpatient (Commercial),Charity Care (Medicare),...,Incentive Payments (Others),Total Contractual Allowances (Medicaid),Patient Days of Swing Bed (Medicare),Discharges of DPU (Others),Other Operating Revenue,Non Operating Expense,Hospital Name,Hospital Short Name,Type,Critical Access
0,6920003,2007,1,2702.0,23366.0,0.0,4373.0,0,0.0,0.0,...,0.0,33963338.0,0,156,3531158.0,0.0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False
1,6920003,2007,2,2894.0,24675.0,0.0,4497.0,0,0.0,0.0,...,0.0,39446639.0,0,138,3766613.0,0.0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False
2,6920003,2007,3,2706.0,24595.0,0.0,4530.0,0,0.0,0.0,...,0.0,33742710.0,0,123,3805685.0,0.0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False
3,6920003,2007,4,2669.0,23755.0,0.0,4210.0,0,0.0,0.0,...,0.0,30356389.0,0,154,3579350.0,0.0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False
4,6920003,2008,1,2695.0,24219.0,0.0,4559.0,0,0.0,0.0,...,0.0,36108352.0,0,158,3723298.0,0.0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False


In [8]:
print((quarterly_sums.isna().sum() / quarterly_sums.shape[0])[(quarterly_sums.isna().sum() / quarterly_sums.shape[0]) > 0.0])

Series([], dtype: float64)


In [9]:
print(list(set(monthly_data.columns).difference(set(quarterly_sums.columns))))

['Month']


In [10]:
quarterly_sums.to_csv(os.path.join(os.getcwd(), 'data', 'input', 'OR_quarterly_aggregation.csv'), index=False)